# 1. Trending
- The **Trending API** and **Topic API** is derived by an emulater named [网易MuMu模拟器](https://mumu.163.com/), and [Charles Proxy](https://www.charlesproxy.com/);
- Here I use the scraped API of CSDN user [考古学家lx](https://me.csdn.net/weixin_43582101) directly;
- It is not an official API [[Reference]](https://blog.csdn.net/weixin_43582101/article/details/103791795).

In [27]:
import os
import json
import requests
import bs4 as bs
import numpy as np
import pandas as pd
import urllib.request
from IPython.core.display import HTML

In [28]:
trending_api = 'https://aweme-hl.snssdk.com/aweme/v1/hot/search/list/?detail_list=1&mac_address=08:00:27:29:D2:F5&os_api=23&device_type=MI%205s&device_platform=android&ssmix=a&iid=92152480453&manifest_version_code=860&dpi=320&uuid=008796750074613&version_code=860&app_name=aweme&version_name=8.6.0&ts=1577932778&openudid=c055533a0591b2dc&device_id=69918538596&resolution=810*1440&os_version=6.0.1&language=zh&device_brand=Xiaomi&app_type=normal&ac=wifi&update_version_code=8602&aid=1128&channel=tengxun_new&_rticket=1577932779592'
re=requests.get(trending_api)
soup=bs.BeautifulSoup(re.content,'html.parser')
last_update = json.loads(soup.text)['data']['active_time']
trending_data = json.loads(soup.text)['data']['word_list']
trend = pd.DataFrame(trending_data)
date=last_update.split(' ')[0]
print('The last update time: '+last_update)

The last update time: 2020-03-02 16:33:47


In [29]:
Word_cover=[]
for i in trend['word_cover']:
    if type(i)==dict:
        Word_cover.append(i['url_list'][0])
    else:
        Word_cover.append(None)

In [30]:
trend = trend.drop(columns = ['word_cover','challenge_id'])

In [31]:
trend.head(3)

,label,position,video_count,group_id,word_type,word,hot_value
0,0,1,39,6647582024964068622,1,甩头发换装,11782203
1,0,2,1,6799126825244038407,1,一个人前后反差可以有多大,8092958
2,0,3,25,6797941205557581067,1,刑释人员离汉抵京调查结果,7571719


In [32]:
def generate_path(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)

In [33]:
# word means the title of the topic
generate_path('./trend')
trend['cover']=['<img src="'+ str(i) + '" width="60" >' for i in Word_cover]
trend_visual = HTML(trend.head(3).to_html(escape=False ,formatters=trend['cover']))
trend.to_html('./trend/trend_'+last_update+'.html', escape=False)
trend.to_csv('./trend/trend_'+last_update+'.csv',encoding = 'utf-8-sig', index = False)
trend_visual

,label,position,video_count,group_id,word_type,word,hot_value,cover
0,0,1,39,6647582024964068622,1,甩头发换装,11782203,
1,0,2,1,6799126825244038407,1,一个人前后反差可以有多大,8092958,
2,0,3,25,6797941205557581067,1,刑释人员离汉抵京调查结果,7571719,


# 2. Video
- With functions as follows, we can get variabels of description, account name, verify, share count, forward count, like count, comment count, and download count of each video;
- And download original videos

In [25]:
def video(video_url,file_name):
    return urllib.request.urlretrieve(video_url,file_name)
def cover(cover_url,file_name):
    return urllib.request.urlretrieve(cover_url,file_name)

In [26]:
def douyin(topic):
    generate_path('./'+topic)
    topic_api='https://aweme-hl.snssdk.com/aweme/v1/hot/search/video/list/?hotword='
    re=requests.get(topic_api+topic)
    soup=bs.BeautifulSoup(re.content,'html.parser')
    data = json.loads(soup.text)
    data = data['aweme_list']
    desc = [info['desc'] for info in data]
    nickname = [info['author']['nickname'] for info in data]
    verify = [info['author']['custom_verify'] for info in data]
    share_count = [info['statistics']['share_count'] for info in data]
    forward_count = [info['statistics']['forward_count'] for info in data]
    like_count = [info['statistics']['digg_count'] for info in data]
    comment_count = [info['statistics']['comment_count'] for info in data]
    download_count = [info['statistics']['download_count'] for info in data]
    cover_url = [info['video']['cover']['url_list'][0] for info in data]
    cover_visual = ['<img src="'+ url + '" width="100" >' for url in cover_url]
    video_url = []
    for info in data:
        video_url.append([i for i in info['video']['download_addr']['url_list'] if 'default' in i][0])
    df=pd.DataFrame({'desc':desc,'nickname':nickname,'verify':verify,
                    'share_count':share_count,'forward_count':forward_count,
                    'like_count':like_count,'comment_count':comment_count,
                    'download_count':download_count,'video_url':video_url,
                    'cover_visual':cover_visual})
    df.to_csv('./'+topic+'/'+topic+'.csv',encoding='utf-8-sig',index=False)
    df.to_html('./'+topic+'/'+topic+'.html',escape=False)
    video_visual = HTML(df.to_html(escape=False ,formatters=df['cover_visual']))
    num=0
    for video_url in df['video_url']:
        video(video_url,'./'+topic+'/'+str(num)+'.mp4')
        num = num+1

In [23]:
print(list(trend['word']))

['甩头发换装', '一个人前后反差可以有多大', '刑释人员离汉抵京调查结果', '新冠肺炎逝者遗体解剖医生刘良', '李易峰本峰的压尾指挑战', '李现刮胡子前后', '刘宇宁黑夜一束光MV', '易烊千玺眼中稳定的工作', '王鹤棣剃光头', '任嘉伦谭松韵合唱六月的雨', '陈数吐槽崔英俊', '周震南童颜的忧伤', '日本多地超市厕纸被抢购一空', '王笑飞腹部扎入3厘米木刺坚持完赛', '法国两名市长确诊新冠肺炎', '王源杂志拍摄花絮', '易烊千玺 GQ封面', '张浩模仿郭德纲发型', '詹姆斯单挑锡安', '夏之光说合肥话', '斯黛拉崔英俊叶东烈直播喊话', '王嘉尔回应被催婚', '詹姆斯超远三分', '欧阳娜娜三姐妹合跳Anysong', '陈瑶收行李打脸岳绮罗', '脱口秀大会斥责白凯南抄袭', '李诞一分钟画李诞', '甜茶 复古油画大片', '蔡徐坤 汤姆猫', '官方回应街道搬走业主认捐蔬菜', '民警硬核警示掉进冰面暗坑', '皇马2比0胜巴萨', 'C罗重返伯纳乌观战国家德比', '戚薇火锅牌泡脚', '霍尊回应下巴脱臼', '计算机模拟疫情期开学的可能后果', '不同地方女生遇到男友前任的反应', '李佳航 老婆说我跳舞像秧歌', '意大利议员戴口罩被嘲怒摔话筒', '黄绮珊为奇迹男孩献唱', '武汉医院的东北话教学', '女孩子笑声的正确打开方式', '宋茜宋威龙好甜', '日本累计确诊新冠肺炎962例', '商业房贷利率可以二选一', '湖北接收社会捐赠资金超130亿元', '武汉儿童医院成立陪护专班', '意大利网红奶奶科普防护指南', '任嘉伦不会滑旱冰']


In [ ]:
# Test with one single topic
douyin('Your topic here')

In [ ]:
# Scrape the whole data of all topics
for i in trend['word']:
    douyin(i)